# **[Main]**

In [2]:
import os
import pandas as pd
import numpy as np
from typing import Dict, Tuple, Optional

CHECK_COLS  = [f"check{i}_value" for i in range(1, 7)]
SERVICE_COLS = [f"service{i}" for i in range(1, 17)]
ID_COL = "menti_seq"
MENTOR_COL = "mento_seq"
DATE_COL = "reg_date"
CHECK_TYPE_COL = "check_type"

# IO Helpers
## mentoring data

In [5]:
def load_f1(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # datetime형식으로 변환 
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    # 숫자형으로 변환 
    for c in [ID_COL, MENTOR_COL, CHECK_TYPE_COL] + CHECK_COLS + SERVICE_COLS:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    sort_cols = [ID_COL]
    # ID별로 그룹화
    # 날짜순 정렬
    if DATE_COL in df.columns:
        sort_cols.append(DATE_COL)
    df = df.sort_values(sort_cols).reset_index(drop=True)
    return df

## E_type

In [9]:
def load_f2(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)
    # 컬럼의 값을 문자열(str)로 변환하고, 거기서 앞뒤 공백을 제거하는 작업
    if "srvy_result" in df.columns:
        df["srvy_result"] = df["srvy_result"].astype(str).str.strip()
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    if ID_COL in df.columns:
        df[ID_COL] = pd.to_numeric(df[ID_COL], errors="coerce")
    return df

## Sequence Building

In [19]:
def build_sequences_from_order(
    f1: pd.DataFrame,
    status_cols = CHECK_COLS,
    service_cols = SERVICE_COLS,
    target_len: int = 40,
    allow_trim_longer: bool=False,
): 
    groups, metas = [], []
    required = [ID_COL] + status_cols + service_cols
    missing = [c for c in required if c not in f1.columns]
    if missing:
        raise KeyError(f"f1 is missing required columns: {missing}")
    # len 40으로 자르기 
    for mid, g in f1.groupby(ID_COL):
        g = g.sort_values(DATE_COL).reset_index(drop=True)
        n = len(g)
        if n == target_len:
            g_use = g.copy()
        elif n > target_len and allow_trim_longer:
            g_use = g.iloc[:target_len].copy()
        else:
            continue

        groups.append((mid, g_use))
        metas.append({ID_COL: mid, "T_i": len(g_use), "first_idx": 0, "last_idx": len(g_use)-1})
# 40일 맞춘거 ID순으로 정렬
    meta = pd.DataFrame(metas).sort_values(ID_COL).reset_index(drop=True)
# menti 수 / S: check length / K: service length 
    # zeros: 0으로 채워진 배열, ones: 1로 채워진 배열  
    N = len(groups); S = len(status_cols); K = len(service_cols)
    status_np = np.zeros((N, target_len, S), dtype=np.float32)
    service_np = np.zeros((N, target_len, K), dtype=np.float32)
    mask_np = np.ones((N, target_len), dtype=bool)

    id2row: Dict[int, int] = {}
    for row_idx, (mid, g) in enumerate(sorted(groups, key=lambda x: x[0])):
        id2row[mid] = row_idx
        st = g[status_cols].fillna(0).to_numpy(dtype=np.float32)
        sv = g[service_cols].fillna(0).to_numpy(dtype=np.float32)
        if st.shape[0] != target_len or sv.shape[0] != target_len:
            raise ValueError(f"unexpected length for menti_seq={mid}: {st.shape[0]} rows")
        status_np[row_idx] = st
        service_np[row_idx] = sv
    return status_np, service_np, mask_np, id2row, meta
    # (N, target_len, S), (N, target_len, K), (N, target_len)

## Labels / outcomes (PHQ->D, P4->I, Lonelinesss->L) using FIRST and LAST

In [21]:
def build_labels_from_f2(
    f2: pd.DataFrame,
    id2row: Dict[int, int],
    alpha=1.0, beta=1.0, gamma=1.0,
):
    """
    For each (menti_seq, srvy_name), use FIRST record as baseline (t=0) and LAST as final (t=T).
    Mapping:
      PHQ         -> D
      P4          -> I
      Lonelinesss -> L
    Returns:
      y_0: (N, 3)  [D_0, I_0, L_0]
      y_T: (N, 3)  [D_T, I_T, L_T]
      r_0: (N, 1)  = αD_0 + βI_0 + γL_0
      r_T: (N, 1)  = αD_T + βI_T + γL_T
    """
    N = len(id2row)
    # np.full은 원하는 값으로 배열을 채워줌 
    y_0 = np.full((N, 3), np.nan, dtype=np.float32)
    y_T = np.full((N, 3), np.nan, dtype=np.float32)
    r_0 = np.full((N, 1), np.nan, dtype=np.float32)
    r_T = np.full((N, 1), np.nan, dtype=np.float32)

    if f2 is None or len(f2) == 0:
        return y_0, y_T, r_0, r_T

    df = f2.copy()
    # 결과를 숫자, 날짜형으로 바꿈 
    if "srvy_result" in df.columns:
        df["srvy_result"] = pd.to_numeric(df["srvy_result"].astype(str).str.strip(), errors="coerce")
    if DATE_COL in df.columns:
        df[DATE_COL] = pd.to_datetime(df[DATE_COL], errors="coerce")
    # 데이터프레임에서 "srvy_name"이라는 컬럼이 있는지 확인한 다음,
    # 그 컬럼의 값이 keep_names 리스트에 포함된 것들만 필터링해서 남기는 거
    keep_names = ["PHQ-9", "P4", "Loneliness"]
    if "srvy_name" not in df.columns:
        return y_0, y_T, r_0, r_T
    df = df[df["srvy_name"].isin(keep_names)]

    # FIRST (baseline)
    if DATE_COL in df.columns:
        first_df = (df.sort_values([ID_COL, "srvy_name", DATE_COL])
                      .groupby([ID_COL, "srvy_name"], as_index=False).first())
    else:
        first_df = df.groupby([ID_COL, "srvy_name"], as_index=False).first()
    # pivot은 행을 열로 바꾸고, 열을 행으로 바꾸는 식
    first_wide = first_df.pivot(index=ID_COL, columns="srvy_name", values="srvy_result")
    first_map = pd.DataFrame(index=first_wide.index)
    first_map["D_0"] = first_wide.get("PHQ-9")
    first_map["I_0"] = first_wide.get("P4")
    first_map["L_0"] = first_wide.get("Loneliness")

    # LAST (final)
    if DATE_COL in df.columns:
        last_df = (df.sort_values([ID_COL, "srvy_name", DATE_COL])
                     .groupby([ID_COL, "srvy_name"], as_index=False).last())
    else:
        last_df = df.groupby([ID_COL, "srvy_name"], as_index=False).last()
    last_wide = last_df.pivot(index=ID_COL, columns="srvy_name", values="srvy_result")
    last_map = pd.DataFrame(index=last_wide.index)
    last_map["D_T"] = last_wide.get("PHQ-9")
    last_map["I_T"] = last_wide.get("P4")
    last_map["L_T"] = last_wide.get("Loneliness")

    # Fill arrays
    for mid in set(list(first_map.index) + list(last_map.index)):
        if mid not in id2row:
            continue
        r = id2row[mid]
        d0 = first_map.loc[mid, "D_0"] if mid in first_map.index else np.nan
        i0 = first_map.loc[mid, "I_0"] if mid in first_map.index else np.nan
        l0 = first_map.loc[mid, "L_0"] if mid in first_map.index else np.nan
        y_0[r] = [d0, i0, l0]
        dT = last_map.loc[mid, "D_T"] if mid in last_map.index else np.nan
        iT = last_map.loc[mid, "I_T"] if mid in last_map.index else np.nan
        lT = last_map.loc[mid, "L_T"] if mid in last_map.index else np.nan
        y_T[r] = [dT, iT, lT]

    r_0 = alpha * y_0[:, [0]] + beta * y_0[:, [1]] + gamma * y_0[:, [2]]
    r_T = alpha * y_T[:, [0]] + beta * y_T[:, [1]] + gamma * y_T[:, [2]]
    return y_0, y_T, r_0, r_T

# End to End 

In [13]:
def preprocess_all(
    f1_path: str,
    f2_path: Optional[str] = None,
    alpha=1.0, beta=1.0, gamma=1.0,
    target_len: int = 40,
    allow_trim_longer: bool = False,
) -> Dict[str, object]: 
    f1 = load_f1(f1_path)
    f2 = load_f2(f2_path) if f2_path else pd.DataFrame()

    status_np, service_np, mask_np, id2row, meta = build_sequences_from_order(
        f1, status_cols=CHECK_COLS, service_cols=SERVICE_COLS,
        target_len=target_len, allow_trim_longer=allow_trim_longer,
    )

    y_0, y_T, r_0, r_T = build_labels_from_f2(f2, id2row, alpha, beta, gamma)

    return dict(
        raw=f1,
        status=status_np,
        service=service_np,
        mask=mask_np,
        id2row=id2row,
        meta=meta,
        y_0=y_0, y_T=y_T,
        r_0=r_0, r_T=r_T,
    )

In [22]:
BASE_DIR = os.getcwd()
F1_PATH = os.path.join(BASE_DIR, "멘토링 일상생활 info 0424.ver.2.csv")   # f1
F2_PATH = os.path.join(BASE_DIR, "E_type_설문결과 모음0424.csv")          # f2

data = preprocess_all(
    f1_path=F1_PATH, f2_path=F2_PATH,
    alpha=1.0, beta=1.0, gamma=1.0,
    target_len=40,
    allow_trim_longer=True,   
)
print("status:", data["status"].shape, "service:", data["service"].shape)  # (N,40,6), (N,40,16)
print("participants:", len(data["id2row"]))
print("y_0 head:\n", data["y_0"][:3])
print("y_T head:\n", data["y_T"][:3])
print("r_0 head:", data["r_0"][:3].ravel())
print("r_T head:", data["r_T"][:3].ravel()) # 다차원 배열을 1차원 배열로 평평하게 펼쳐주는 기능

status: (2209, 40, 6) service: (2209, 40, 16)
participants: 2209
y_0 head:
 [[1. 0. 1.]
 [1. 0. 1.]
 [0. 0. 1.]]
y_T head:
 [[2. 0. 1.]
 [1. 0. 1.]
 [0. 0. 1.]]
r_0 head: [2. 2. 1.]
r_T head: [3. 2. 1.]


# **[Model1]**

## tensorflow 학습용 배치 데이터셋 구성 전처리 파이프라인

In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# 함수 정의에서 : 기호 -> 각 매개변수가 어떤 타입의 값을 받아야 하는지를 명시
def make_tf_dataset(
    status_np: np.ndarray,
    service_np: np.ndarray,
    y_T: np.ndarray,
    r_T: np.ndarray,
    batch_size: int = 32,
    shuffle: bool = True,
): 
    # astype은 데이터 타입을 변환할 때 사용하는 메서드
    X_seq = np.concatenate([status_np, service_np], axis=-1).astype(np.float32)

    # None이나 결측값 처리 
    if y_T is None:
        y_clean = np.zeros((len(X_seq), 3), np.float32)
        y_w = np.zeros((len(X_seq),), np.float32)
    else:
        y_clean = np.nan_to_num(y_T, nan=0.0).astype(np.float32)
        # 평탄화 
        y_w = np.isfinite(y_T).any(axis=1).astype(np.float32)
# 평탄화 진행한 이유
# 각 샘플에 유효한 값이 하나라도 있는지 확인해서, 해당 샘플을 학습에 사용할지 말지를 결정하는 마스크 벡터 만드는 것

    if r_T is None:
        r_clean = np.zeros((len(X_seq), 1), np.float32)
        r_w = np.zeros((len(X_seq),), np.float32)
    else:
        r_clean = np.nan_to_num(r_T, nan=0.0).astype(np.float32)
        r_w = np.isfinite(r_T).ravel().astype(np.float32)
        
    ds = tf.data.Dataset.from_tensor_slices(
        (X_seq, (y_clean, r_clean), (y_w, r_w))
    )
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X_seq), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds 

## Custom Layer: Consistency loss for Keras 3.x

In [26]:
class ConsistencyLoss(layers.Layer):
    def __init__(self, alpha=1.0, beta=1.0, gamma=1.0, mu=0.25, name="consistency_loss", **kwargs):
    # **kwargs: 이렇게 쓰면, ConsistencyLoss 클래스를 만들 때 전달된 추가적인 키워드 인자들을 부모 클래스인 layers.Layer에게 그대로 넘겨줄 수 있음 
        super().__init__(name=name, **kwargs)
        self.alpha = float(alpha)
        self.beta = float(beta)
        self.gamma = float(gamma)
        self.mu = float(mu)
        self._consistency_tracker = keras.metrics.Mean(name="consistency_mse")
        # self._consistency_tracker = keras.metrics.Mean(name="consistency_mse")
    @property
    def metrics(self):
        return [self._consistency_tracker]

    def call(self, inputs, training=None):
        y_pred, r_pred = inputs
        d_hat = y_pred[:, 0:1]
        i_hat = y_pred[:, 1:2]
        l_hat = y_pred[:, 2:3]
        r_from_y = self.alpha * d_hat + self.beta * i_hat + self.gamma * l_hat
        # tensorflow에서 평균값을 계산하는 함수. reduce는 다차원 배열을 하나의 값으로 줄이는 연산
        # mean은 평균이니까 전체 또는 특정 축의 평균을 구하는 함수 
        loss = tf.reduce_mean(tf.square(r_pred - r_from_y))
        self.add_loss(self.mu * loss)
        self._consistency_tracker.update_state(loss)
        return r_pred

    def compute_output_shape(self, input_shape):
        return input_shape[1]

## model1 _ GRU(Gated Recurrent Unit: RNN의 구조를 개선한 게이트 기반 RNN)
(정보를 얼마나 기억할지, 얼마나 잊을지를 게이트로 조절함)
- 긴 시퀀스도 잘 기억함
- LSTM보다 구조가 간단해서 계산 효율이 좋음
- 학습 속도 빠름 

In [54]:
def build_model_1 (
    timesteps: int = 40,
    in_features: int = 22, # 6 + 16
    d_model: int = 128,
    dropout: float = 0.2,
    r_consistency_mu: float = 0.25,
    alpha: float = 1.0, beta: float = 1.0, gamma: float = 1.0,
):
    
    x = layers.Input(shape=(timesteps, in_features), name="x_seq")
    h = layers.Masking(mask_value=0.0)(x)
# 입력값 중 0.0인 부분은 무시하고 GRU가 학습하지 않도록 함
    h = layers.GRU(d_model, return_sequences=True)(h)
# 각 타임스텝마다 은닉 상태 출력
    h = layers.Dropout(dropout)(h)
# 과적합 방지를 위해 일부 뉴런을 랜덤하게 제거 
    h = layers.GRU(d_model)(h)
# 전체 시퀀스를 하나의 벡터로 요약
    h = layers.Dropout(dropout)(h)
# 첫 번째 GRU: 시퀀스를 유지해서 시간 흐름 학습
# 두 번째 GRU: 전체 시퀀스를 요약해서 최종 표현을 만듦 
# - Masking: 결측값이나 패딩된 부분을 무시해서 학습 정확도 향상.
# - Dropout: 일반화 성능을 높이기 위한 정규화 기법.

    y_out = layers.Dense(64, activation="relu")(h)
    y_out = layers.Dropout(dropout)(y_out) # 학습 안정화
    y_out = layers.Dense(3, name="y_out")(y_out)

    r_raw = layers.Dense(64, activation="relu")(h)
    r_raw = layers.Dropout(dropout)(r_raw)
    r_raw = layers.Dense(1, name="r_out_raw")(r_raw)

    r_out = ConsistencyLoss(alpha=alpha, beta=beta, gamma=gamma, mu=r_consistency_mu, name="consistency")([y_out, r_raw])
    r_out = layers.Identity(name="r_out")(r_out)
# Identity: 아무것도 하지 않는 레이어 _ 그대로 출력으로 전달

    model = keras.Model(inputs=x, outputs=[y_out, r_out], name="Model1_GRU")
# keras.Model은 위의 layer들을 하나로 묶어주는 것 
# 다중 출력 모델을 학습시키기 위한 설정
# y_out, r_out이라는 두 개의 출력 각각에 대해 손실함수 지정, 손실 가중치 지정, 평가 지표 지정 
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss=["mse", "mse"],
        loss_weights = [1.0, 1.0],
        metrics=[
            [keras.metrics.MeanAbsoluteError(name="mae_y"),
            keras.metrics.RootMeanSquaredError(name="rmse_y")],
            [keras.metrics.MeanAbsoluteError(name="mae_r"),
            keras.metrics.RootMeanSquaredError(name="rmse_r")],
        ],
    )
    return model

In [52]:
def _train_val_split_indices(N: int, val_split: float, seed: int=42):
    # val_split: 검증 데이터 비율
    rng = np.random.default_rng(seed)
    idx = np.arange(N)
    rng.shuffle(idx)
# 너무 작은 데이터셋일 경우
# - 검증 샘플이 0개가 되면 최소 1개로 보정
# - 학습 샘플이 0개가 되면 최소 1개로 보정
    n_val = int(np.floor(N * val_split))
    if val_split > 0 and N >= 2 and n_val == 0:
        n_val = 1
    if N - n_val == 0 and N >= 2:
        n_val = N-1

    val_idx = idx[:n_val]
    tr_idx = idx[n_val:]
    return tr_idx, val_idx

In [56]:
def train_model_1(
    data: dict,
    alpha: float = 1.0, beta: float = 1.0, gamma: float = 1.0,
    lambda_r: float = 1.0,
    mu_consistency: float = 0.25,
    batch_size: int = 32,
    epochs: int = 50,
    val_split: float = 0.2,
    seed: int = 42,
    save_dir: str = "ckpt_model1",
): 
    tf.keras.utils.set_random_seed(seed)
    os.makedirs(save_dir, exist_ok=True)

    status = data["status"]
    service = data["service"]
    y_T = data["y_T"]
    r_T = data.get("r_T", None)
    # "r_T"라는 키가 있으면 → 해당 값을 반환
    # "r_T"라는 키가 없으면 → None을 반환

    N, T = status.shape[0], status.shape[1]
    in_features = status.shape[2] + service.shape[2]

    tr_idx, val_idx = _train_val_split_indices(N, val_split, seed)

    def take(a, ids):
        return a[ids] if (a is not None and len(ids) > 0) else None

    ds_train = make_tf_dataset(
        take(status, tr_idx), take(service, tr_idx),
        take(y_T, tr_idx), take(r_T, tr_idx),
        batch_size=batch_size, shuffle=True
    )
    ds_val = None
    if val_idx is not None and len(val_idx) > 0:
        ds_val = make_tf_dataset(
            take(status, val_idx), take(service, val_idx),
            take(y_T, val_idx), take(r_T, val_idx),
            batch_size=batch_size, shuffle=False
        )
    model = build_model_1(
        timesteps=T,
        in_features=in_features,
        d_model=128,
        dropout=0.2,
        r_consistency_mu=mu_consistency,
        alpha=alpha, beta=beta, gamma=gamma,
    )

    # 딥러닝에서의 컴파일: 모델이 어떻게 학습할지 정의하는 단계
    # λ_r adjust하기 위해서 re compile
    model.compile(
        optimizer=keras.optimizers.Adam(1e-3),
        loss=["mse", "mse"],
        loss_weights=[1.0, float(lambda_r)],
        metrics=[
            [keras.metrics.MeanAbsoluteError(name="mae_y"),
            keras.metrics.RootMeanSquaredError(name="rmse_y")],
            [keras.metrics.MeanAbsoluteError(name="mae_r"),
            keras.metrics.RootMeanSquaredError(name="rmse_r")],
        ],
    )

    # ds_val: 검증 데이터셋, monitor_metric: 모니터링할 지표 이름 
    # ds_val이 없으면 → "loss" 사용 (훈련 손실만 모니터링)
    # ds_val이 있으면 → "val_loss" 사용 (검증 손실을 기준으로 모니터링)
    monitor_metric = "loss" if ds_val is None else "val_loss"
    callbacks = [
        # 가장 성능 좋은 모델을 저장  
        keras.callbacks.ModelCheckpoint(
            filepath=os.path.join(save_dir, "best.keras"),
            monitor=monitor_metric,
            save_best_only=True,
            save_weights_only=False,
            verbose=1,
        ),
        # 성능이 정체되면 학습률을 줄임
        keras.callbacks.ReduceLROnPlateau(
            monitor=monitor_metric, factor=0.5, patience=5, min_lr=1e-5, verbose=1
        ),
        # min_lr: 학습률이 아무리 줄어들어도 최소한 이 값 이하로는 떨어지지 않게 하겠다는 설정
        # verbose = 1: 진행 막대(progress bar) 형태로 출력 (0: 아무것도 출력하지 않음, 2: 배치 단위 로그 출력)
        # 성능이 더 이상 좋아지지 않으면 학습을 멈춤
        keras.callbacks.EarlyStopping(
            monitor=monitor_metric, patience=10, restore_best_weights=True, verbose=1
        ),
    ]

    history = model.fit(
        ds_train,
        validation_data=ds_val,
        epochs=epochs,
        callbacks=callbacks,
        verbose=1,
    )
    
    return model, history 

# Entry Point

In [57]:
if __name__ == "__main__":

    model, history = train_model_1(
        data,
        alpha=1.0, beta=1.0, gamma=1.0,
        lambda_r=1.0,
        mu_consistency=0.25,
        batch_size=32,
        epochs=50,
        val_split=0.2,
        seed=42,
        save_dir="ckpt_model1",
    )

    print(model.summary())

Epoch 1/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - consistency_mse: 0.2376 - loss: 1.6795 - r_out_loss: 1.2791 - r_out_mae_r: 0.8566 - r_out_rmse_r: 1.1272 - y_out_loss: 0.3408 - y_out_mae_y: 0.4330 - y_out_rmse_y: 0.5828
Epoch 1: val_loss improved from inf to 0.92248, saving model to ckpt_model1\best.keras
56/56 ━━━━━━━━━━━━━━━━━━━━ 17s 119ms/step - consistency_mse: 0.2365 - loss: 1.6748 - r_out_loss: 1.2754 - r_out_mae_r: 0.8553 - r_out_rmse_r: 1.1257 - y_out_loss: 0.3400 - y_out_mae_y: 0.4324 - y_out_rmse_y: 0.5821 - val_consistency_mse: 0.0690 - val_loss: 0.9225 - val_r_out_loss: 0.6837 - val_r_out_mae_r: 0.6273 - val_r_out_rmse_r: 0.8340 - val_y_out_loss: 0.2184 - val_y_out_mae_y: 0.3616 - val_y_out_rmse_y: 0.4680 - learning_rate: 0.0010
Epoch 2/50
56/56 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step - consistency_mse: 0.1255 - loss: 1.2968 - r_out_loss: 0.9962 - r_out_mae_r: 0.7567 - r_out_rmse_r: 1.0033 - y_out_loss: 0.2691 - y_out_mae_y: 0.3743 - y_out_rmse_y: 0.5193
Epoch 2: val_loss imp

Model: "Model1_GRU"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ x_seq (InputLayer)            │ (None, 40, 22)            │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ not_equal_2 (NotEqual)        │ (None, 40, 22)            │               0 │ x_seq[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ masking_2 (Masking)           │ (None, 40, 22)            │               0 │ x_seq[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ any_2 (Any)                   │ (None, 40)                │               0 │ not_equal_2[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gru_4 (GRU)                   │ (None, 40, 128)           │          58,368 │ masking_2[0][0],           │
│                               │                           │                 │ any_2[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_7 (Dropout)           │ (None, 40, 128)           │               0 │ gru_4[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gru_5 (GRU)                   │ (None, 128)               │          99,072 │ dropout_7[0][0],           │
│                               │                           │                 │ any_2[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_8 (Dropout)           │ (None, 128)               │               0 │ gru_5[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 64)                │           8,256 │ dropout_8[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_4 (Dense)               │ (None, 64)                │           8,256 │ dropout_8[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_9 (Dropout)           │ (None, 64)                │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_10 (Dropout)          │ (None, 64)                │               0 │ dense_4[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ y_out (Dense)                 │ (None, 3)                 │             195 │ dropout_9[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ r_out_raw (Dense)             │ (None, 1)                 │              65 │ dropout_10[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ consistency (ConsistencyLoss) │ (None, 1)                 │               0 │ y_out[0][0],               │
│                               │                           │                 │ r_out_raw[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ r_out (Identity)              │ (None, 1)                 │               

 Total params: 522,638 (1.99 MB)

 Trainable params: 174,212 (680.52 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 348,426 (1.33 MB)

None


## 성능 check

In [60]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
try:
    from sklearn.metrics import root_mean_squared_error
    _HAS_RMSE_FN = True
except Exception:
    _HAS_RMSE_FN = False


X = np.concatenate([data["status"], data["service"]], axis=-1).astype(np.float32)
y_pred, r_pred = model.predict(X, batch_size=256, verbose=0)

r_true = data.get("r_T")
mask_r = np.isfinite(r_true).ravel()
r_true_nz = r_true[mask_r].ravel()
r_pred_nz = r_pred[mask_r].ravel()

mae_r = mean_absolute_error(r_true_nz, r_pred_nz) if r_true is not None else np.nan
if _HAS_RMSE_FN:
    from sklearn.metrics import root_mean_squared_error
    rmse_r = root_mean_squared_error(r_true_nz, r_pred_nz)
else:
    rmse_r = float(np.sqrt(mean_squared_error(r_true_nz, r_pred_nz)))
r2_r = r2_score(r_true_nz, r_pred_nz) if r_true_nz.size > 1 else np.nan
print(f"[r_out] MAE={mae_r:.4f} RMSE={rmse_r:.4f} R^2={r2_r:.4f}")
y_true = data.get("y_T")  # (N,3)
if y_true is not None:
    names = ["D", "I", "L"]
    for j, name in enumerate(names):
        mask_y = np.isfinite(y_true[:, j])
        yt = y_true[mask_y, j]
        yp = y_pred[mask_y, j]
        mae  = mean_absolute_error(yt, yp)
        if _HAS_RMSE_FN:
            rmse = root_mean_squared_error(yt, yp)
        else:
            rmse = float(np.sqrt(mean_squared_error(yt, yp)))
        r2    = r2_score(yt, yp) if yt.size > 1 else np.nan
        print(f"[y_out:{name}] MAE={mae:.4f}  RMSE={rmse:.4f}  R²={r2:.4f}")
else:
    print("[y_out] y_T labels not available; skipped.")

[r_out] MAE=0.6848 RMSE=0.9057 R^2=0.0873
[y_out:D] MAE=0.5399  RMSE=0.6501  R²=0.1130
[y_out:I] MAE=0.0735  RMSE=0.3059  R²=-0.0080
[y_out:L] MAE=0.3695  RMSE=0.4177  R²=-0.0263
